### The True State

In order to help users understand what is going on, it is necessary to be able to pull out the true state of the network at any time. This is obtained by calling the get_agent_state method and passing in 'True'. Since this observation is huge, we will focus on 'User0'.

In [1]:
import inspect
from pprint import pprint
from CybORG import CybORG
from CybORG.Agents import B_lineAgent

path = str(inspect.getfile(CybORG))
path = path[:-10] + '/Shared/Scenarios/Scenario1b.yaml'

env = CybORG(path, 'sim',agents={'Red':B_lineAgent})

results = env.reset(agent='Blue')

true_state = env.get_agent_state('True')
pprint(true_state['User0'])
print(76*'-')

{'Interface': [{'IP Address': IPv4Address('127.0.0.1'),
                'Interface Name': 'lo',
                'Subnet': IPv4Network('127.0.0.0/8')},
               {'IP Address': IPv4Address('10.0.46.54'),
                'Interface Name': 'eth0',
                'Subnet': IPv4Network('10.0.46.48/28')}],
 'Processes': [{'Connections': [{'Transport Protocol': <TransportProtocol.UNKNOWN: 1>,
                                 'local_address': IPv4Address('0.0.0.0'),
                                 'local_port': 22}],
                'Known Path': <Path.UNKNOWN: 1>,
                'Known Process': <ProcessName.SSHD: 7>,
                'PID': 3368,
                'PPID': 5956,
                'Path': 'C:\\Program Files\\OpenSSH\\usr\\sbin',
                'Process Name': 'sshd.exe',
                'Process Type': <ProcessType.SSH: 2>,
                'Username': 'sshd_server'},
               {'Connections': [{'Transport Protocol': <TransportProtocol.UNKNOWN: 1>,
                    

In order to better visualise this, we will import the true_obs_to_table function, which converts this observation into a human-readable table.

In [2]:
from CybORG.Agents.Wrappers.TrueTableWrapper import true_obs_to_table

true_table = true_obs_to_table(true_state,env)
print(true_table)

Scanned column likely inaccurate.
+-----------------+--------------+-------------+-------+---------+------------+
|      Subnet     |  IP Address  |   Hostname  | Known | Scanned |   Access   |
+-----------------+--------------+-------------+-------+---------+------------+
|  10.0.108.0/28  |  10.0.108.6  |   Defender  | False |  False  |    None    |
|  10.0.108.0/28  |  10.0.108.8  | Enterprise0 | False |  False  |    None    |
|  10.0.108.0/28  |  10.0.108.5  | Enterprise1 | False |  False  |    None    |
|  10.0.108.0/28  |  10.0.108.7  | Enterprise2 | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.116 |   Op_Host0  | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.120 |   Op_Host1  | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.114 |   Op_Host2  | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.118 |  Op_Server0 | False |  False  |    None    |
|  10.0.46.48/28  |  10.0.46.54  |    User0    |  True |  False  | Privileged |
|  10.

As red moves throughout the network, we can thus see its progress. Note how the 'User1' exploit gets root immediately.

In [3]:
for i in range(3):
    env.step()
    true_state = env.get_agent_state('True')
    true_table = true_obs_to_table(true_state,env)
    print(env.get_last_action('Red'))
    print(true_table)
    print(76*'-')

Scanned column likely inaccurate.
DiscoverRemoteSystems 10.0.46.48/28
+-----------------+--------------+-------------+-------+---------+------------+
|      Subnet     |  IP Address  |   Hostname  | Known | Scanned |   Access   |
+-----------------+--------------+-------------+-------+---------+------------+
|  10.0.108.0/28  |  10.0.108.6  |   Defender  | False |  False  |    None    |
|  10.0.108.0/28  |  10.0.108.8  | Enterprise0 | False |  False  |    None    |
|  10.0.108.0/28  |  10.0.108.5  | Enterprise1 | False |  False  |    None    |
|  10.0.108.0/28  |  10.0.108.7  | Enterprise2 | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.116 |   Op_Host0  | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.120 |   Op_Host1  | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.114 |   Op_Host2  | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.118 |  Op_Server0 | False |  False  |    None    |
|  10.0.46.48/28  |  10.0.46.54  |    User0    |  

Blue team then restores 'User1' and we can see Red's access is gone.

In [4]:
from CybORG.Shared.Actions import Restore
action = Restore(hostname='User1',session=0,agent='Blue')
env.step(action=action,agent='Blue')

true_state = env.get_agent_state('True')
true_table = true_obs_to_table(true_state,env)
print(true_table)

Scanned column likely inaccurate.
+-----------------+--------------+-------------+-------+---------+------------+
|      Subnet     |  IP Address  |   Hostname  | Known | Scanned |   Access   |
+-----------------+--------------+-------------+-------+---------+------------+
|  10.0.108.0/28  |  10.0.108.6  |   Defender  | False |  False  |    None    |
|  10.0.108.0/28  |  10.0.108.8  | Enterprise0 | False |  False  |    None    |
|  10.0.108.0/28  |  10.0.108.5  | Enterprise1 | False |  False  |    None    |
|  10.0.108.0/28  |  10.0.108.7  | Enterprise2 | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.116 |   Op_Host0  | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.120 |   Op_Host1  | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.114 |   Op_Host2  | False |  False  |    None    |
| 10.0.127.112/28 | 10.0.127.118 |  Op_Server0 | False |  False  |    None    |
|  10.0.46.48/28  |  10.0.46.54  |    User0    |  True |  False  | Privileged |
|  10.

## Other Debugging Tools

CybORG has a host of other tools to help understand the agent state. We have already see the get_observation method.

In [9]:
env.reset()
env.step()

red_obs = env.get_observation('Red')
pprint(red_obs)

blue_obs = env.get_observation('Blue')
pprint(blue_obs)
print(76*'-')

{'10.0.214.129': {'Interface': [{'IP Address': IPv4Address('10.0.214.129'),
                                 'Subnet': IPv4Network('10.0.214.128/28')}]},
 '10.0.214.132': {'Interface': [{'IP Address': IPv4Address('10.0.214.132'),
                                 'Subnet': IPv4Network('10.0.214.128/28')}]},
 '10.0.214.135': {'Interface': [{'IP Address': IPv4Address('10.0.214.135'),
                                 'Subnet': IPv4Network('10.0.214.128/28')}]},
 '10.0.214.138': {'Interface': [{'IP Address': IPv4Address('10.0.214.138'),
                                 'Subnet': IPv4Network('10.0.214.128/28')}]},
 '10.0.214.141': {'Interface': [{'IP Address': IPv4Address('10.0.214.141'),
                                 'Subnet': IPv4Network('10.0.214.128/28')}]},
 'success': <TrinaryEnum.TRUE: 1>}
{'success': <TrinaryEnum.TRUE: 1>}
----------------------------------------------------------------------------


We have also seen the get_last_action method. Note that since we haven't specified an action or agent for blue it defaults to Sleep.

In [10]:
red_action = env.get_last_action('Red')
print(red_action)

blue_action = env.get_last_action('Blue')
print(blue_action)

DiscoverRemoteSystems 10.0.214.128/28
Sleep


The get_action_space method allows us to get the action space of any agent.

In [21]:
red_action_space = env.get_action_space('Red')
print(list(red_action_space))

blue_action_space = env.get_action_space('Blue')
print(list(blue_action_space.keys()))

['action', 'subnet', 'ip_address', 'session', 'username', 'password', 'process', 'port', 'target_session', 'agent', 'hostname']
['action', 'subnet', 'ip_address', 'session', 'username', 'password', 'process', 'port', 'target_session', 'agent', 'hostname']


The get_ip_map method allows us to see which hostnames are associated with each ip. This is something that is known to Blue, but not to Red at the beginning of the game so be careful to not abuse it.

In [22]:
env.get_ip_map()

{'Enterprise0': IPv4Address('10.0.36.246'),
 'Enterprise1': IPv4Address('10.0.36.242'),
 'Enterprise2': IPv4Address('10.0.36.245'),
 'Defender': IPv4Address('10.0.36.247'),
 'Op_Server0': IPv4Address('10.0.226.251'),
 'Op_Host0': IPv4Address('10.0.226.247'),
 'Op_Host1': IPv4Address('10.0.226.246'),
 'Op_Host2': IPv4Address('10.0.226.252'),
 'User0': IPv4Address('10.0.214.129'),
 'User1': IPv4Address('10.0.214.141'),
 'User2': IPv4Address('10.0.214.138'),
 'User3': IPv4Address('10.0.214.135'),
 'User4': IPv4Address('10.0.214.132')}

The get_rewards method allows us to see the rewards for all agents.

In [25]:
env.get_rewards()

{'Blue': 0.0, 'Green': 0.0, 'Red': 0.0}

Finally, the set_seed method allows us to set a random seed.

In [27]:
env.set_seed(100)